In [7]:
import torch
from random import shuffle
import numpy as np
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import pandas as pd

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [8]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
LABELS = [    
    "JUMPING",
    "JUMPING_JACKS",
    "BOXING",
    "WAVING_2HANDS",
    "WAVING_1HAND",
    "CLAPPING_HANDS"

] 
DATASET_PATH = "poses/"

X_train_path = "shuffled_x_train_data"
X_test_path = DATASET_PATH + "X_test.txt"

y_train_path = "shuffled_y_train_data"
y_test_path = DATASET_PATH + "Y_test.txt"

# Hyper-parameters 
# input_size = 784 # 28x28
num_classes = 6
num_epochs = 10
train_batch_size = 125
test_batch_size = 1
learning_rate = 0.00001

input_size = 36
sequence_length = 32
hidden_size = 200
num_layers = 3

In [9]:
device

device(type='cpu')

In [3]:
column_names = [  "j0_x",  "j0_y", "j1_x", "j1_y" , "j2_x", "j2_y", "j3_x", "j3_y", "j4_x", "j4_y", "j5_x", "j5_y", "j6_x", "j6_y", "j7_x", "j7_y", "j8_x", "j8_y", "j9_x", "j9_y", "j10_x", "j10_y", "j11_x", "j11_y", "j12_x", "j12_y", "j13_x", "j13_y", 'j14_x', "j14_y", "j15_x", "j15_y", "j16_x", "j16_y", "j17_x", "j17_y" ]
# x_train_data = pd.read_csv(X_train_path, sep=",", names=column_names, header=None, dtype=np.float32)
shuffled_x_train_data = pd.read_csv(X_train_path, sep="\t", dtype=np.float32)
x_test_data = pd.read_csv(X_test_path, sep=",", names=column_names, header=None, dtype=np.float32)
# y_train_data = p|d.read_csv(y_train_path, names=["labels"], dtype=np.int_)
shuffled_y_train_data = pd.read_csv(y_train_path, sep="\t", dtype=np.int_)
y_test_data = pd.read_csv(y_test_path, names=["labels"], dtype=np.int_)

In [4]:
def normalize(data):
    for i, column in enumerate(data):
        if i % 2 == 0:
            data[column] = data[column] / 640
        else:
            data[column] = data[column] / 480
    return data

In [5]:
def unnormalize(data):
    for i, column in enumerate(data):
        if i % 2 == 0:
            data[column] = data[column] * 640
        else:
            data[column] = data[column] * 480
    return data

In [5]:
# normalize(x_train_data)
normalize(x_test_data).head()


,j0_x,j0_y,j1_x,j1_y,j2_x,j2_y,j3_x,j3_y,j4_x,j4_y,...,j13_x,j13_y,j14_x,j14_y,j15_x,j15_y,j16_x,j16_y,j17_x,j17_y
0,0.480608,0.339533,0.499006,0.429050,0.458230,0.426417,0.445991,0.521419,0.433775,0.597352,...,0.527416,0.858235,0.476553,0.333940,0.492905,0.333894,0.0,0.0,0.515238,0.336773
1,0.480573,0.339540,0.499003,0.429056,0.458214,0.426448,0.445981,0.521417,0.433775,0.594800,...,0.527414,0.858202,0.476514,0.333948,0.492866,0.333892,0.0,0.0,0.513323,0.336781
2,0.478591,0.339481,0.498986,0.429035,0.456236,0.426415,0.445988,0.524065,0.433777,0.597444,...,0.527422,0.858233,0.472530,0.333890,0.490823,0.333875,0.0,0.0,0.513273,0.336835
3,0.478508,0.336883,0.498959,0.426423,0.456230,0.426333,0.445983,0.521427,0.431887,0.597506,...,0.527445,0.858206,0.470505,0.331300,0.488778,0.331308,0.0,0.0,0.513252,0.336785
4,0.476562,0.336775,0.497027,0.426348,0.454209,0.423727,0.445997,0.521408,0.431880,0.597565,...,0.527423,0.858217,0.470452,0.328621,0.486744,0.331202,0.0,0.0,0.513173,0.336777


In [6]:
# y_train_data = y_train_data - 1
y_test_data = y_test_data - 1
y_test_data

,labels
0,0
1,0
2,0
3,0
4,0
...,...
5746,5
5747,5
5748,5
5749,5


In [10]:
def shuffle_data(x_data, y_data):
    length = y_data.shape[0]
    shuffled_indexes = [i for i in range(length)]
    shuffle(shuffled_indexes)
    shuffled_y_data = pd.DataFrame()
    shuffled_x_data = pd.DataFrame()
    for i, index in enumerate(shuffled_indexes): 
        shuffled_y_data = shuffled_y_data.append(y_data.iloc[[index]], ignore_index=True)
        shuffled_x_data = shuffled_x_data.append(x_data.iloc[index*32:index*32+32], ignore_index=True)
        if (i % 1000 == 0):
            print(f"{i} done")
    return shuffled_x_data, shuffled_y_data

In [12]:
shuffled_x_test_data, shuffled_y_test_data = shuffle_data(x_test_data, y_test_data)

0 done
1000 done
2000 done
3000 done
4000 done
5000 done


In [13]:
shuffled_x_test_data.to_csv("shuffled_x_test_data.csv", sep='\t', index=False)
shuffled_y_test_data.to_csv("shuffled_y_test_data.csv", sep='\t', index=False)

In [10]:
# Fully connected neural network with one hidden layer
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTM, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        # -> x needs to be: (batch_size, seq, input_size)
        
        # or:100
        #self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        #self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        # Set initial hidden states (and cell states for LSTM)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        
        # x: (n, 28, 28), h0: (2, n, 128)
        
        # Forward propagate RNN
        # out, _ = self.rnn(x, h0)  
        # or:
        out, _ = self.lstm(x, (h0,c0))  
        
        # out: tensor of shape (batch_size, seq_length, hidden_size)
        # out: (n, 28, 128)
        
        # Decode the hidden state of the last time step
        # out = out[:, -1, :]
        # out: (n, 128)
         
        out = self.fc(out)
        out = out[:, -1, :]
        # out: (n, 10)
        return out

In [16]:
model = LSTM(input_size, hidden_size, num_layers, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [7]:
def chunker(x_seq, y_seq, seq_size, batch_size):
    for batch_pos in range(0, len(y_seq), batch_size):
        x_batch = list()
        for pos in range(batch_size):
            # print(x_seq.iloc[pos*seq_size:pos*seq_size + seq_size])
            x_batch.append(x_seq.iloc[pos*seq_size:pos*seq_size + seq_size].values)
        
        yield torch.tensor(x_batch), torch.flatten(torch.tensor(y_seq.iloc[batch_pos:batch_pos + batch_size].values))
        

In [8]:
y_train = shuffled_y_train_data.iloc[:int(shuffled_y_train_data.shape[0]*0.9 - 362)]
x_train = shuffled_x_train_data.iloc[:int(y_train.shape[0]*32)]
x_test = shuffled_x_train_data.iloc[x_train.shape[0]:]
y_test = shuffled_y_train_data.iloc[y_train.shape[0]:]
print(x_train.shape[0]/32)
print(y_train.shape)
print(x_test.shape[0]/32)
print(y_test.shape)

20000.0
(20000, 1)
2625.0
(2625, 1)


In [17]:
# Train the model
n_total_steps = int(y_train.shape[0] / 200)

for epoch in range(num_epochs):
    correct = 0
    for i, (x_batch, y_batch) in enumerate(chunker(x_train, y_train, sequence_length, 200)):  
        # origin shape: [N, 1, 28, 28]
        # resized: [N, 28, 28]
        y_batch = y_batch.to(device)
        x_batch = x_batch.to(device)
        optimizer.zero_grad()
        # Forward pass
        outputs = model(x_batch)
        # print(outputs.shape)
        
        loss = criterion(outputs, y_batch)
        # print(loss.item())
        # Backward and optimize
        
        loss.backward()
        optimizer.step()

        # max returns (value ,index)
        _, predicted = torch.max(outputs.data, 1)
        # print(predicted.shape)
        correct += (predicted == y_batch).sum().item()
        if (i+1) % 10 == 0:
            
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
    acc = 100.0 * correct / y_train.shape[0]
    print (f'Accuracy: {acc} %')
    # Test the model
    # In test phase, we don't need to compute gradients (for memory efficiency)
    correct = 0
    with torch.no_grad():
        correct = 0
        for i, (x_batch, y_batch) in enumerate(chunker(x_test, y_test, sequence_length, 125)):  
            # origin shape: [N, 1, 28, 28]
            # resized: [N, 28, 28]
            y_batch = y_batch.to(device)
            x_batch = x_batch.to(device)
            outputs = model(x_batch)
            
            # max returns (value ,index)
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == y_batch).sum().item()
        acc = 100.0 * correct / y_test.shape[0]
        print(f'Accuracy: {acc} %')

Epoch [1/10], Step [10/100], Loss: 1.7981
Epoch [1/10], Step [20/100], Loss: 1.7932
Epoch [1/10], Step [30/100], Loss: 1.7918
Epoch [1/10], Step [40/100], Loss: 1.7940
Epoch [1/10], Step [50/100], Loss: 1.7925
Epoch [1/10], Step [60/100], Loss: 1.7857
Epoch [1/10], Step [70/100], Loss: 1.7930
Epoch [1/10], Step [80/100], Loss: 1.7901
Epoch [1/10], Step [90/100], Loss: 1.7876
Epoch [1/10], Step [100/100], Loss: 1.7894
Accuracy: 16.825 %
Accuracy: 23.2 %
Epoch [2/10], Step [10/100], Loss: 1.7852
Epoch [2/10], Step [20/100], Loss: 1.7835
Epoch [2/10], Step [30/100], Loss: 1.7814
Epoch [2/10], Step [40/100], Loss: 1.7741
Epoch [2/10], Step [50/100], Loss: 1.7789
Epoch [2/10], Step [60/100], Loss: 1.7738
Epoch [2/10], Step [70/100], Loss: 1.7742
Epoch [2/10], Step [80/100], Loss: 1.7765
Epoch [2/10], Step [90/100], Loss: 1.7704
Epoch [2/10], Step [100/100], Loss: 1.7643
Accuracy: 22.23 %
Accuracy: 23.2 %
Epoch [3/10], Step [10/100], Loss: 1.7551
Epoch [3/10], Step [20/100], Loss: 1.7626
Epo

KeyboardInterrupt: 